In [179]:
%matplotlib inline

In [180]:
#Import packages
import requests
import json
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib import mlab
import seaborn as sns
import pandas as pd
from scipy.stats import poisson
import os
import re
sns.set_palette("husl")

In [181]:
acc = "NZ_CP0007399"
apiurl = "http://phaster.ca/phaster_api?acc=NZ_CP007399" 
r=requests.get(apiurl)
d=r.json()['summary']

In [182]:
m=re.search("REGION\s*REGION_LENGTH",d)
m2 = re.search("---------------------------------*\n ",d)

summary = d[d.index(m2.group(0))+len(m2.group(0)):]
header = d[d.index(m.group(0)):d.index(m2.group(0))]

summary = summary.split()
header = header.split()

In [183]:
for i in range(int(len(summary)/len(header))):
    score = summary[i*len(header)+2]
    position = summary[i*len(header)+4]
    print(score, position)

intact(100) 924880-942808
questionable(85) 943667-967155
intact(107) 1484949-1499404
intact(100) 3211688-3230714
questionable(80) 3958150-4001095
intact(150) 4964557-4989828


In [184]:
header

['REGION',
 'REGION_LENGTH',
 'COMPLETENESS(score)',
 'SPECIFIC_KEYWORD',
 'REGION_POSITION',
 'TRNA_NUM',
 'TOTAL_PROTEIN_NUM',
 'PHAGE_HIT_PROTEIN_NUM',
 'HYPOTHETICAL_PROTEIN_NUM',
 'PHAGE+HYPO_PROTEIN_PERCENTAGE',
 'BACTERIAL_PROTEIN_NUM',
 'ATT_SITE_SHOWUP',
 'PHAGE_SPECIES_NUM',
 'MOST_COMMON_PHAGE_NAME(hit_genes_count)',
 'FIRST_MOST_COMMON_PHAGE_NUM',
 'FIRST_MOST_COMMON_PHAGE_PERCENTAGE',
 'GC_PERCENTAGE']

In [163]:
position

'4491197-4548442'